In [48]:
import pandas as pd
import numpy as np
from glob import glob

In [231]:
def calculate_totals(df) :
    df['Mean_system_time_check_in'] = df[['Mean_serve_time_check_in', 'Mean_wait_time_check_in']].sum(axis = 1)
    df['Mean_system_time_refine_complaint'] = df[['Mean_serve_time_refine_complaint', 'Mean_wait_time_refine_complaint']].sum(axis = 1)
    df['Mean_system_time_exam'] = df[['Mean_serve_time_exam_flow_staff', 'Mean_wait_time_exam']].sum(axis = 1)
    df['Mean_system_time_follow_up'] = df[['Mean_serve_time_follow_up', 'Mean_wait_time_follow_up']].sum(axis = 1)
    df['Mean_system_time_checkout'] = df[['Mean_serve_time_checkout', 'Mean_wait_time_checkout']].sum(axis = 1)
    df['Total_service_time'] = df[['Mean_serve_time_check_in', 'Mean_serve_time_refine_complaint', 'Mean_serve_time_exam', 'Mean_serve_time_follow_up', 'Mean_serve_time_checkout']].sum(axis = 1)
    df['Total_wait_time'] = df[['Mean_wait_time_check_in', 'Mean_wait_time_refine_complaint', 'Mean_wait_time_exam', 'Mean_wait_time_follow_up', 'Mean_wait_time_checkout']].sum(axis = 1)
    df['Total_system_time'] = df[['Total_service_time', 'Total_wait_time']].sum(axis = 1)
    # Calculate throughputs
    df['Thruput_check_in'] = df['Served_check_in'] / df['Arrivals_check_in']
    df['Thruput_refine_complaint'] = df['Served_refine_complaint'] / df['Arrivals_refine_complaint']
    df['Thruput_exam'] = df['Served_exam'] / df['Arrivals_exam']
    df['Thruput_follow_up'] = df['Served_follow_up'] / df['Arrivals_follow_up']
    df['Thruput_checkout'] = df['Served_checkout'] / df['Arrivals_checkout']
    df['Thruput_total'] = df['Served_checkout'] / df['Arrivals_check_in']
    return df

In [232]:
fpaths = glob('Results of sims/Sims/200614_offload_staff*.csv')

In [233]:
sorted(fpaths)

['Results of sims/Sims/200614_offload_staff10209_offload00.csv',
 'Results of sims/Sims/200614_offload_staff10209_offload05.csv',
 'Results of sims/Sims/200614_offload_staff10209_offload10.csv',
 'Results of sims/Sims/200614_offload_staff10209_offload15.csv',
 'Results of sims/Sims/200614_offload_staff10209_offload20.csv',
 'Results of sims/Sims/200614_offload_staff10209_offload25.csv',
 'Results of sims/Sims/200614_offload_staff10209_offload30.csv',
 'Results of sims/Sims/200614_offload_staff10219_offload00.csv',
 'Results of sims/Sims/200614_offload_staff10219_offload05.csv',
 'Results of sims/Sims/200614_offload_staff10219_offload10.csv',
 'Results of sims/Sims/200614_offload_staff10219_offload15.csv',
 'Results of sims/Sims/200614_offload_staff10219_offload20.csv',
 'Results of sims/Sims/200614_offload_staff10219_offload25.csv',
 'Results of sims/Sims/200614_offload_staff10219_offload30.csv',
 'Results of sims/Sims/200614_offload_staff10229_offload00.csv',
 'Results of sims/Sims/20

In [234]:
data = pd.DataFrame()
for f in fpaths :
    _ = pd.read_csv(f)
    data = pd.concat([data, _])

In [235]:
data = data.drop('Unnamed: 0', axis = 1)
data['Staffing'] = data['Staffing'].replace('10/20/09', '10-20-9')
data['Staffing'] = data['Staffing'].replace('10/21/09', '10-21-9')

In [236]:
data = calculate_totals(data)

In [237]:
data.shape

(210000, 44)

In [238]:
control_fpaths = glob('Results of sims/Sims/200614_offload_control*.csv')

In [239]:
sorted(control_fpaths)

['Results of sims/Sims/200614_offload_control_10209.csv',
 'Results of sims/Sims/200614_offload_control_10219.csv',
 'Results of sims/Sims/200614_offload_control_10229.csv']

In [240]:
control = pd.DataFrame()
for f in control_fpaths :
    _ = pd.read_csv(f, index_col = 0)
    _ = _.T.reset_index()
    _ = _.drop('index', axis = 1)
    control = pd.concat([control, _])

In [241]:
control['Staffing'] = control['Staffing'].replace('10/20/09', '10-20-9')
control['Staffing'] = control['Staffing'].replace('10/21/09', '10-21-9')

In [242]:
control.shape

(30000, 41)

In [243]:
cols_add = list(set(data.columns) - set(control.columns))

In [244]:
for c in cols_add :
    control[c] = np.nan

In [245]:
ordered_cols = list(data.columns)

In [246]:
control = control[ordered_cols]

In [247]:
data = pd.concat([data, control])

In [248]:
data.shape

(240000, 44)

In [249]:
data.groupby(['Staffing', 'Offload']).size()

Staffing  Offload
10-20-9   0.0        10000
          0.05       10000
          0.1        10000
          0.15       10000
          0.2        10000
          0.25       10000
          0.3        10000
          Control    10000
10-21-9   0.0        10000
          0.05       10000
          0.1        10000
          0.15       10000
          0.2        10000
          0.25       10000
          0.3        10000
          Control    10000
10-22-9   0.0        10000
          0.05       10000
          0.1        10000
          0.15       10000
          0.2        10000
          0.25       10000
          0.3        10000
          Control    10000
dtype: int64

In [250]:
data.to_csv('Results of sims/Sims/200614_offload_alldata.csv')